In [1]:
import math
import numpy as np
import pandas as pd
import scipy.io as sio
import scipy.optimize as opt
import time
from search import *

In [8]:
class RubikCube(Problem):
    """
    Rubik Cube division: 6 colors, 9 elements each, 3 elements per line
          Line h.:     top   ||   middle  ||  bottom
             pos :  l | m | r   l | m | r   l | m | r  (left | middle | right)
    Red color    = (01, 02, 03, 04, 05, 06, 07, 08, 09)   math.floor(n/10) == 0, element of the Red color
    Blue color   = (11, 12, 13, 14, 15, 16, 17, 18, 19)   math.floor(n/10) == 1, element of the Blue color 
    Green color  = (21, 22, 23, 24, 25, 26, 27, 28, 29)   math.floor(n/10) == 2, element of the Green color 
    Orange color = (31, 32, 33, 34, 35, 36, 37, 38, 39)   math.floor(n/10) == 3, element of the Orange color 
    Yellow color = (41, 42, 43, 44, 45, 46, 47, 48, 49)   math.floor(n/10) == 4, element of the Yellow color 
    White color  = (51, 52, 53, 54, 55, 56, 57, 58, 59)   math.floor(n/10) == 5, element of the White color
    
    All in one array:                     Face distribution:       (How faces connect to each other)
    (01, 02, 03, 04, 05, 06, 07, 08, 09,  face 0: pos 00 to 08, left => 1 R, right => 2 R, up => 4 C, down => 5 C
     11, 12, 13, 14, 15, 16, 17, 18, 19,  face 1: pos 09 to 17, left => 3 R, right => 0 R, up => 4 R, down => 5 R
     21, 22, 23, 24, 25, 26, 27, 28, 29,  face 2: pos 18 to 26, left => 0 R, right => 3 R, up => 4 R, down => 5 R
     31, 32, 33, 34, 35, 36, 37, 38, 39,  face 3: pos 27 to 35, left => 2 R, right => 1 R, up => 4 C, down => 5 C
     41, 42, 43, 44, 45, 46, 47, 48, 49,  face 4: pos 36 to 44, left => 1 C, right => 2 C, up => 3 C, down => 0 C
     51, 52, 53, 54, 55, 56, 57, 58, 59)  face 5: pos 45 to 53, left => 1 C, right => 2 C, up => 0 C, down => 3 C
                                          Every 9 pos in the array is a face
    """
    def getCube(self,state):
        face0 = np.reshape(state[:9],(3,3),"C")
        face1 = np.reshape(state[9:9*2],(3,3),"C")
        face2 = np.reshape(state[9*2:9*3],(3,3),"C")
        face3 = np.reshape(state[9*3:9*4],(3,3),"C")
        face4 = np.reshape(state[9*4:9*5],(3,3),"C")
        face5 = np.reshape(state[9*5:9*6],(3,3),"C")
        return [face0, face1, face2, face3, face4, face5]
    
    def __init__(self, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 
                             33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59)):
        cube = self.getCube(goal) #transform the state into a tridimensional matrix, easyer to manage
        print("Goal state:")
        self.paint_Rubik_Cube(cube)
        print("---------------------------------------------------------")
        """Let's alter our solved Rubik Cube randomly so we can solve it"""
        print("Alter process:")
        self.movements = ['UP', 'DOWN', 'LEFT', 'RIGHT'] #The movements we can make
        self.faces_connections = [[[4,"C",0,0],[5,"C",0,0],[1,"R",0,0],[2,"R",0,0]], #by self.movements order
                                  [[4,"R",1,0],[5,"R",0,1],[3,"R",0,0],[0,"R",0,0]], #[Number of the next face affected by the movement,
                                  [[4,"R",0,1],[5,"R",1,0],[0,"R",0,0],[3,"R",0,0]], #If the movement will affect a Row or a Column,
                                  [[4,"C",1,1],[5,"C",1,1],[2,"R",0,0],[1,"R",0,0]], #If you need to invert the positions of elements,
                                  [[3,"C",1,1],[0,"C",0,0],[1,"C",1,0],[2,"C",0,1]], #If you need to invert the number of the line]
                                  [[0,"C",0,0],[3,"C",1,1],[1,"C",0,1],[2,"C",1,0]]]
        iterations = np.random.randint(1,10) #moves we are going to make to modify it
        for i in range (iterations):
            face = np.random.randint(5) #pick a random face
            line = np.random.randint(2) #a random line, row or column, depends of the movement
            move = self.movements[np.random.randint(3)] #and a random movement to do
            print("Move nº ",i+1,": Face => ",face," Line => ",line," Move => ",move)
            cube = self.make_a_move(face, line, move, cube)  
            #self.paint_Rubik_Cube(cube)
        print("---------------------------------------------------------")
        print("Initial state:")
        self.paint_Rubik_Cube(cube)
        initial = np.reshape(cube,(6*3*3),"C") #reshape back to state array
        super().__init__(initial, goal)

    def make_a_move(self, face, line, move, cube):
        if move == 'UP' or move == 'DOWN': #the direction of the movement change the elements that form the line
            selected = [cube[face][0][line],cube[face][1][line],cube[face][2][line]] #the initial elements to move 
            if line == 2: #if the line is not the middle one, the adjacent face rotates with it
                adjacent_face = self.faces_connections[face][3][0]
                if move =='UP':
                    cube[adjacent_face] = np.rot90(cube[adjacent_face], k=-1)
                else:
                    cube[adjacent_face] = np.rot90(cube[adjacent_face], k=1)
            elif line == 0:
                adjacent_face = self.faces_connections[face][2][0]
                if move =='UP':
                    cube[adjacent_face] = np.rot90(cube[adjacent_face], k=1)
                else:
                    cube[adjacent_face] = np.rot90(cube[adjacent_face], k=-1)       
        else:
            selected = cube[face][line]
            if line == 2:
                adjacent_face = self.faces_connections[face][1][0]
                if move =='RIGHT':
                    cube[adjacent_face] = np.rot90(cube[adjacent_face], k=-1)
                else:
                    cube[adjacent_face] = np.rot90(cube[adjacent_face], k=1)
            elif line == 0:
                adjacent_face = self.faces_connections[face][0][0]
                if move =='RIGHT':
                    cube[adjacent_face] = np.rot90(cube[adjacent_face], k=1)
                else:
                    cube[adjacent_face] = np.rot90(cube[adjacent_face], k=-1)
                
        previous_face = face #since directions are relative to each face, we need to know from where we came to know where to go   
        for n in range(4): #each movement 4 faces are altered
            if move == 'UP': #the sense of the movement tells which face is to be modified
                next_face = self.faces_connections[face][0]
            elif move == 'DOWN':
                next_face = self.faces_connections[face][1]
            elif move == 'LEFT':
                next_face = self.faces_connections[face][2]
            else:
                next_face = self.faces_connections[face][3]
                
            if next_face[1] == "R": #Will a Row or a Column be affected by the movement?
                if next_face[3] == 1 and line != 1: #Do you need to invert the line number?
                    if line == 0:
                        line = 2
                    else:
                        line = 0
                #Save the elements that will be overwritten because they will be the next to move
                next_selected = [cube[next_face[0]][line][0],cube[next_face[0]][line][1],cube[next_face[0]][line][2]] 
                if next_face[2] == 0: #Do you need to invert positions of the elements?
                    cube[next_face[0]][line] = selected #Then move the elements to the new face
                else:
                    cube[next_face[0]][line] = [selected[2],selected[1],selected[0]]
            else:
                if next_face[3] == 1 and line != 1: #Do you need to invert the line number?
                    if line == 0:
                        line = 2
                    else:
                        line = 0
                next_selected = [cube[next_face[0]][0][line],cube[next_face[0]][1][line],cube[next_face[0]][2][line]]
                if next_face[2] == 0:
                    cube[next_face[0]][0][line] = selected[0]
                    cube[next_face[0]][1][line] = selected[1]
                    cube[next_face[0]][2][line] = selected[2]
                else:
                    cube[next_face[0]][0][line] = selected[2]
                    cube[next_face[0]][1][line] = selected[1]
                    cube[next_face[0]][2][line] = selected[0]
                    
            #Now we actualize for the next iteration
            selected = next_selected
            previous_face = face
            face = next_face[0]
            if previous_face == self.faces_connections[face][0][0]: #The move is updated to keep the sense and direction
                move = 'DOWN'
            elif previous_face == self.faces_connections[face][1][0]:
                move = 'UP'
            elif previous_face == self.faces_connections[face][2][0]:
                move = 'RIGHT'
            else:
                move = 'LEFT'
            #self.paint_Rubik_Cube(cube)
        
        return cube
            
    def actions(self, state): #let the search know what can do to solve the cube
        #for each face there are four posible movements, and each move can choose between three lines
        #[face, line, move]
        possible_actions = [[0,0,self.movements[0]],[0,1,self.movements[0]],[0,2,self.movements[0]],
                            [0,0,self.movements[1]],[0,1,self.movements[1]],[0,2,self.movements[1]],
                            [0,0,self.movements[2]],[0,1,self.movements[2]],[0,2,self.movements[2]],
                            [0,0,self.movements[3]],[0,1,self.movements[3]],[0,2,self.movements[3]],
                            [1,0,self.movements[0]],[1,1,self.movements[0]],[1,2,self.movements[0]],
                            [1,0,self.movements[1]],[1,1,self.movements[1]],[1,2,self.movements[1]],
                            [1,0,self.movements[2]],[1,1,self.movements[2]],[1,2,self.movements[2]],
                            [1,0,self.movements[3]],[1,1,self.movements[3]],[1,2,self.movements[3]],
                            [2,0,self.movements[0]],[2,1,self.movements[0]],[2,2,self.movements[0]],
                            [2,0,self.movements[1]],[2,1,self.movements[1]],[2,2,self.movements[1]],
                            [2,0,self.movements[2]],[2,1,self.movements[2]],[2,2,self.movements[2]],
                            [2,0,self.movements[3]],[2,1,self.movements[3]],[2,2,self.movements[3]],
                            [3,0,self.movements[0]],[3,1,self.movements[0]],[3,2,self.movements[0]],
                            [3,0,self.movements[1]],[3,1,self.movements[1]],[3,2,self.movements[1]],
                            [3,0,self.movements[2]],[3,1,self.movements[2]],[3,2,self.movements[2]],
                            [3,0,self.movements[3]],[3,1,self.movements[3]],[3,2,self.movements[3]],
                            [4,0,self.movements[0]],[4,1,self.movements[0]],[4,2,self.movements[0]],
                            [4,0,self.movements[1]],[4,1,self.movements[1]],[4,2,self.movements[1]],
                            [4,0,self.movements[2]],[4,1,self.movements[2]],[4,2,self.movements[2]],
                            [4,0,self.movements[3]],[4,1,self.movements[3]],[4,2,self.movements[3]],
                            [5,0,self.movements[0]],[5,1,self.movements[0]],[5,2,self.movements[0]],
                            [5,0,self.movements[1]],[5,1,self.movements[1]],[5,2,self.movements[1]],
                            [5,0,self.movements[2]],[5,1,self.movements[2]],[5,2,self.movements[2]],
                            [5,0,self.movements[3]],[5,1,self.movements[3]],[5,2,self.movements[3]],]       
        return possible_actions
        
    def result(self, state, action):
        new_state = list(state)
        cube = self.getCube(new_state)
        face = action[0]
        line = action[1]
        move = action[2]
        cube = self.make_a_move(face, line, move, cube)
        new_state = np.reshape(cube,(6*3*3),"C")
        return tuple(new_state)
    
    def goal_test(self, state): #check if we reach the goal
        return np.all(state == self.goal)
 
    def h(self, node): #heuristic
        return np.sum(np.abs(np.asarray(node.state) - np.asarray(self.goal))) #Manhattan distance

    def paint_Rubik_Cube(self, cube):
        s = ""
        for i in range(3):
            s += "         \t["
            for j in range(3):
                x = cube[4][i][j]
                if math.floor(x/10) == 0:
                    s += "0"
                if j < 2:
                    s += str(x) + "," 
                else:
                    s += str(x)
            s += "]\n"
        s += "\n"
        aux = [cube[1],cube[0],cube[2],cube[3]]
        for i in range(3):
            for n in range(4):
                s += "["   
                for j in range(3):
                    x = aux[n][i][j]
                    if math.floor(x/10) == 0:
                        s += "0"
                    if j < 2:
                        s += str(x) + "," 
                    else:
                        s += str(x)
                s += "]\t"
            s += "\n"
        s += "\n"
        for i in range(3):
            s += "         \t["
            for j in range(3):
                x = cube[5][i][j]
                if math.floor(x/10) == 0:
                    s += "0"
                if j < 2:
                    s += str(x) + "," 
                else:
                    s += str(x)
            s += "]\n"
        print(s)

In [9]:
cube = RubikCube()

Goal state:
         	[41,42,43]
         	[44,45,46]
         	[47,48,49]

[11,12,13]	[01,02,03]	[21,22,23]	[31,32,33]	
[14,15,16]	[04,05,06]	[24,25,26]	[34,35,36]	
[17,18,19]	[07,08,09]	[27,28,29]	[37,38,39]	

         	[51,52,53]
         	[54,55,56]
         	[57,58,59]

---------------------------------------------------------
Alter process:
Move nº  1 : Face =>  3  Line =>  1  Move =>  LEFT
Move nº  2 : Face =>  4  Line =>  1  Move =>  UP
Move nº  3 : Face =>  0  Line =>  0  Move =>  DOWN
---------------------------------------------------------
Initial state:
         	[39,02,43]
         	[16,25,46]
         	[33,08,49]

[17,04,11]	[41,52,03]	[21,22,23]	[31,48,57]	
[18,05,12]	[44,55,26]	[34,35,36]	[14,45,54]	
[19,06,13]	[47,58,09]	[27,28,29]	[37,42,51]	

         	[01,38,53]
         	[24,15,56]
         	[07,32,59]



In [10]:
print("A* search results:", astar_search(cube).solution())

A* search results: [[0, 0, 'UP'], [0, 1, 'DOWN'], [0, 1, 'RIGHT']]
